In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


Download the pytorch-lightning/wandb packages

In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning

In [ ]:
!pip install wandb

In [5]:
import wandb
wandb.login(relogin=True)
wandb.__version__

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'0.15.4'

Change the directory to where the source was uploaded

In [1]:
cd /content/drive/MyDrive/Project/step4-5

/content/drive/MyDrive/Project/step4-5


Import some underlying functions

In [2]:
import torch
import sys
from torch import nn
import torch.optim as optim

import time
import importlib
import warnings

Specify the path of Datasets

In [3]:
DATA_DIR = '/content/drive/MyDrive/DATASET/'

Set the argumentation of hyperparameters

In [4]:
import dataset
from utils import parse_args, modify_command_options, check_args
from run import run_experiment

Pretraining

In [5]:
# Pretraining

# Fw_task:
# sourceonly ------direct domain adaptation
# fda_inv -----fourier domain adaptation

# DATASET Partition -----A/B
# Clients type: uniform/heterogeneous

# !!!set random_seed to None if it needs to drop the DDP
parser = parse_args()
args = parser.parse_args(['--framework', 'federated','--fw_task', 'fda_inv',
                          '--source_dataset', 'gta5', '--target_dataset','cityscapes',
                          '--clients_type','heterogeneous','--partition', 'B',
                          '--model', 'bisenetv2'])

args.name = 'gta5_cityscapes_pretrain'
args.device_ids = [0]
args.random_seed = None
args.wandb_entity = 'feddrive2023'

args.ignore_warnings = True
args.save_samples = 3
args.color_jitter = True

args.n_images_per_style = 1000
args.clients_per_round = -1
if args.fw_task == 'fda_inv':
    args.fda_b = 2
    args.n_images_per_style = 20


args.pretrain = True
args.num_source_epochs = 15
args.output_aux = True
args.num_epochs = 1
args.num_rounds = 1500

args.hnm = True
args.batch_size = 16
args.test_batch_size = 1

args.server_opt = 'SGD'
args.weight_decay = 0.0005
args.momentum = 0.9

args.lr = 0.04
args.lr_policy = 'poly'
args.lr_power = 0.9
args.sourceonly_loss= 'lovasz_entropy_div'

args.plot_interval=10
args.print_interval=1
args.server_eval_interval=1
args.server_test_interval=1
args.test_interval=1

args = modify_command_options(args)

print(args)
check_args(args)

Namespace(framework='federated', fw_task='fda_inv', local_rank=0, device_ids=[0], random_seed=None, source_dataset='gta5', target_dataset='cityscapes', partition='B', double_dataset=False, quadruple_dataset=False, cv2=False, random_flip=False, color_jitter=True, gaussian_blur=False, clients_type='heterogeneous', centr_fda_ft_uda=False, disable_batch_norm=False, batch_norm_round_0=False, hp_filtered=False, pretrain=True, n_images_per_style=20, fda_L=0.01, fda_b=1, fda_size=(1024, 512), train_source_round_interval=None, lr_factor_server_retrain=1.0, num_source_epochs_factor_retrain=1.0, num_source_steps_retrain=None, source_style_to_source=False, style_only_train=False, sourceonly_loss='lovasz_entropy_div', fda_loss='selftrain', lambda_selftrain=1, lambda_entropy=0.005, teacher_step=1, teacher_kd_step=-1, teacher_upd_step=False, teacher_kd_upd_step=False, teacher_kd_mult_factor=-1.0, teacher_kd_mult_step=5, ignore255_kdloss=False, count_classes_teacher_step=-1, temperature=0.01, weights_

Execute the pertraining experiment

In [ ]:
sys.setrecursionlimit(10000)

if args.ignore_warnings:
    warnings.filterwarnings("ignore")

start = time.time()

run_experiment(args)

end = time.time()

secs = end - start
mins = secs // 60
secs %= 60
hours = mins // 60
mins %= 60

if args.local_rank == 0:
    print(f"Elapsed time: {int(hours)}h, {int(mins)}min, {round(secs, 2)}s")

UDA federated semantic segmentation

In [10]:
#Unsupervised domain adaptation using pretrained model

# Fw_task:
# sourceonly ------training the model based on directly pertrained model
# fda_inv -----training the model with pertrained model which adopts fourier domain adaptation

# DATASET Partition -----A/B
# Clients type: uniform/heterogeneous

# !!!set random_seed to None if it needs to drop the DDP
# !!!specify your wand_id correspondingly if want to upload your aforementioned best checkpoint 

parser = parse_args()
args = parser.parse_args(['--framework', 'federated','--fw_task', 'fda_inv',
                          '--source_dataset', 'gta5', '--target_dataset','cityscapes',
                          '--clients_type','heterogeneous','--partition', 'B',
                          '--model', 'bisenetv2'])

args.name = 'gta5_cityscapes_UDA'
args.device_ids = [0]
args.random_seed = None
args.wandb_entity = 'feddrive2023'

args.ignore_warnings = True
args.save_samples = 3
args.color_jitter = True

args.load_SourceOnly = True
args.load_SourceOnly_id = '21dp89oq'
args.load_SourceOnly_best = True
args.sourceonly_loss= 'selftrain_div'

if args.fw_task == 'fda_inv':
    args.load_FDA = True
    args.load_FDA_id = 'bmx0lpkp'
    args.load_FDA_best = True
    args.fda_b = 1
    args.n_images_per_style = 20
    args.fda_loss= 'selftrain_div'

args.clients_per_round = 5
args.teacher_step = -1  #-1 never update
                        #1 update at beginnning of each round
                        #5 updata every 5 rounds

args.teacher_kd_step = -1
args.swa_teacher_start = -1

args.output_aux = True
args.num_epochs = 3
args.num_rounds = 60
args.num_source_epochs = 10

args.hnm = True
args.batch_size = 8
args.test_batch_size = 1

args.server_opt = 'SGD'
args.weight_decay = 0.0005
args.momentum = 0.9

args.lr = 0.04
args.lr_policy = 'poly'
args.lr_power = 0.9
args.lr_fed = 0.0007  #fw_task.client----train:
                      #lr=args.lr_fed if not self.source_client else None

args.lambda_entropy = 1
args.lambda_selftrain = 1


args.plot_interval=10
args.print_interval=1
args.server_test_interval=1
args.test_interval=1
args.server_eval_interval = 1000000
args.eval_interval = 1000000

args = modify_command_options(args)

print(args)
check_args(args)

Namespace(framework='federated', fw_task='fda_inv', local_rank=0, device_ids=[0], random_seed=None, source_dataset='gta5', target_dataset='cityscapes', partition='B', double_dataset=False, quadruple_dataset=False, cv2=False, random_flip=False, color_jitter=True, gaussian_blur=False, clients_type='heterogeneous', centr_fda_ft_uda=False, disable_batch_norm=False, batch_norm_round_0=False, hp_filtered=False, pretrain=False, n_images_per_style=20, fda_L=0.01, fda_b=1, fda_size=(1024, 512), train_source_round_interval=None, lr_factor_server_retrain=1.0, num_source_epochs_factor_retrain=1.0, num_source_steps_retrain=None, source_style_to_source=False, style_only_train=False, sourceonly_loss='selftrain_div', fda_loss='selftrain_div', lambda_selftrain=1, lambda_entropy=1, teacher_step=-1, teacher_kd_step=-1, teacher_upd_step=False, teacher_kd_upd_step=False, teacher_kd_mult_factor=-1.0, teacher_kd_mult_step=5, ignore255_kdloss=False, count_classes_teacher_step=-1, temperature=0.01, weights_lov

Execute the UDA experiment

In [ ]:
sys.setrecursionlimit(10000)

if args.ignore_warnings:
    warnings.filterwarnings("ignore")

start = time.time()

run_experiment(args)

end = time.time()

secs = end - start
mins = secs // 60
secs %= 60
hours = mins // 60
mins %= 60

if args.local_rank == 0:
    print(f"Elapsed time: {int(hours)}h, {int(mins)}min, {round(secs, 2)}s")